In [2]:
import json
import requests
import pandas as pd
import boto3
from fractions import Fraction

In [35]:
# --------------------------------------------------- CÓDIGO DE LAMBDA LAUREATES --------------------------------------------

url = 'http://api.nobelprize.org/2.1/laureates?ID={}'
    
#TODO Pasar parametro por API
laureate_id = 467
    
info_laureate = requests.get(url.format(laureate_id))
    
resultApi=info_laureate.json()
    
laureateDict=resultApi['laureates'][0]

if 'gender' in laureateDict:   
    fullName=laureateDict['fullName']['en']
    fileName=laureateDict['fileName']
    gender=laureateDict['gender']
    birth=laureateDict['birth']['date']
else:
    fullName='None'
    fileName=laureateDict['fileName']
    gender='None'
    birth='None'
    
dfParquet=pd.DataFrame(columns=['id','fullName','fileName','gender','birth'])
    
dfParquet=dfParquet.append({
    
    'id':laureateDict['id'],
    'fullName': fullName,
    'fileName':fileName,
    'gender':gender,
    'birth':birth
    
} ,ignore_index=True)
     

s3=boto3.client('s3')
    
content=dfParquet.to_parquet()
prueba=dfParquet.to_json()
s3.put_object(Body=content, Bucket='alvaro-munoz-nobelprice', Key='raw/laureate/'+ str(laureate_id)  +'.parquet')

C:\Users\steve\AppData\Local\Temp\ipykernel_13412\1689382244.py:27: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfParquet=dfParquet.append({


{'ResponseMetadata': {'RequestId': '9MVF0Y16DF3Q5EKH',
  'HostId': 'QwBrTQQ6hQPtYkbaUtzwQny6Ga8msrdDMmDSS1T5R4KJ5sYh7zUf5mHbHCINKj1AGeEP+ocf+GE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'QwBrTQQ6hQPtYkbaUtzwQny6Ga8msrdDMmDSS1T5R4KJ5sYh7zUf5mHbHCINKj1AGeEP+ocf+GE=',
   'x-amz-request-id': '9MVF0Y16DF3Q5EKH',
   'date': 'Mon, 08 Aug 2022 12:11:17 GMT',
   'etag': '"319594dece2ff297bb812061052f7bd2"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"319594dece2ff297bb812061052f7bd2"'}

In [8]:
# --------------------------------------------------- CÓDIGO DE LAMBDA NOBEL PRIZES --------------------------------------------

event_category = "phy"
event_year = 2018
url = 'http://api.nobelprize.org/2.1/nobelPrize/{}/{}' 
    
resultApi = requests.get(url.format(event_category,event_year))

laureates_id=[]
laureates_portion=[]
resultApi=resultApi.json()
resultDict=resultApi[0]
try:
    prueba2=resultDict['laureates']
    dateAwarded=resultDict['dateAwarded']
    for i in range(len(prueba2)):
        laureates_id.append(prueba2[i]['id'])
        laureates_portion.append(prueba2[i]['portion'])
    print(arrId)
except:
    laureates_id=None
    dateAwarded=None
    laureates_portion=None
        
dfNobelPrize = pd.DataFrame(columns=['awardYear','category','categoryFullName','dateAwarded','prizeAmount','prizeAmountAdjusted','id','portion'] )
    
    
dfNobelPrize=dfNobelPrize.append({
    
                'id': laureates_id,
                'portion': laureates_portion,
                'awardYear': resultDict['awardYear'],
                'category': resultDict['category']['en'],
                'categoryFullName': resultDict['categoryFullName']['en'],
                'dateAwarded': dateAwarded,
                'prizeAmount': resultDict['prizeAmount'],
                'prizeAmountAdjusted': resultDict['prizeAmountAdjusted']
                    
        }, ignore_index=True)
    
dfNobelPrize=dfNobelPrize.set_index('id')
    
    
    
s3=boto3.client('s3')
    
content=dfNobelPrize.to_parquet()
s3.put_object(Body=content, Bucket='alvaro-munoz-nobelprice', Key='raw/nobelPrize/'+ event_category +'-'+ str(event_year)  +'.parquet')
    
        
# return {
#     'statusCode': 200,
#     'body': json.dumps(event)
# }

['960', '961', '962']


C:\Users\steve\AppData\Local\Temp\ipykernel_5796\1425278466.py:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfNobelPrize=dfNobelPrize.append({


{'ResponseMetadata': {'RequestId': 'JPQVD3F2QCR0X8DB',
  'HostId': '3YKPZrGMHdp0dJWluRQAVzRltKSHJnbI8Hu+xPBhxGZwByGN+E+uSsNyFcVHQ8e4U0br/TfkYhY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '3YKPZrGMHdp0dJWluRQAVzRltKSHJnbI8Hu+xPBhxGZwByGN+E+uSsNyFcVHQ8e4U0br/TfkYhY=',
   'x-amz-request-id': 'JPQVD3F2QCR0X8DB',
   'date': 'Fri, 05 Aug 2022 12:08:00 GMT',
   'etag': '"ab4e0d5adcd1f56ceaf9ff6b0c723f24"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"ab4e0d5adcd1f56ceaf9ff6b0c723f24"'}

In [ ]:
# ------------------------ LECTURA DESDE EL S3 -------------------------------

df_nobel_s3 = pd.read_parquet(path='s3://alvaro-munoz-nobelprice/raw/nobelPrize/')
df_laureates_s3 = pd.read_parquet(path='s3://alvaro-munoz-nobelprice/raw/laureate/')

df_laureates_s3.head(60)
df_nobel_s3.head(60)

In [29]:
#  ------------------------------- LECTURA DE DATOS DE LOS FICHEROS EN LOCAL - TRATAMIENTO DE DATOS ----------------------------------


laureates_id=[]
laureates_portion=[]

try:
    info = resultDic['laureates']
    dateAward = resultDic['dateAwarded']
    for i in range(len(info)):
        laureates_id.append(info[i]['id'])
        laureates_portion.append(info[i]['portion'])
except:
    laureates_id = None
    laureates_portion = None
    dateAward = None


df_laureates = pd.read_parquet('laureates-1901-2019.parquet')
new_types = {
    'id':'int',
    'fullName':'string',
    'fileName':'string',
    'gender':'string',
    'birth':'string'
}
df_laureates=df_laureates.astype(new_types)
df_laureates.dropna(subset = ['fullName'], inplace = True)
df_laureates=df_laureates.explode('id')
df_laureates.head(60)


new_types = {
    'awardYear':'string',
    'category':'string',
    'categoryFullName':'string',
    'dateAwarded':'string',
    'prizeAmount':'int',
    'prizeAmountAdjusted':'int',
    'laureates_id':'int',
}
df_nobel_prizes = pd.read_parquet('nobelPrizes-1901-2019.parquet')

df_nobel_prizes.dropna(subset = ['dateAwarded'], inplace = True)
df_nobel_prizes=df_nobel_prizes.explode('laureates_id')

df_nobel_prizes=df_nobel_prizes.astype(new_types)

mix_df = df_nobel_prizes.join(df_laureates, on='laureates_id', how='inner')
mix_df.head(60)

,awardYear,category,categoryFullName,dateAwarded,prizeAmount,prizeAmountAdjusted,laureates_id,laureates_portion,id,fullName,fileName,gender,birth
2,1901,Peace,The Nobel Peace Prize,1901-12-10,150782,8722510,462,"[1/2, 1/2]",406,David Baltimore,baltimore,male,1938-03-07
2,1901,Peace,The Nobel Peace Prize,1901-12-10,150782,8722510,463,"[1/2, 1/2]",407,Renato Dulbecco,dulbecco,male,1914-02-22
7,1902,Peace,The Nobel Peace Prize,1902-12-10,141847,8205634,464,"[1/2, 1/2]",408,Howard Martin Temin,temin,male,1934-12-10
7,1902,Peace,The Nobel Peace Prize,1902-12-10,141847,8205634,465,"[1/2, 1/2]",249,William N. Lipscomb,lipscomb,male,1919-12-09
12,1903,Peace,The Nobel Peace Prize,1903-12-10,141358,7936836,466,[1],688,Milton Friedman,friedman,male,1912-07-31
17,1904,Peace,The Nobel Peace Prize,1904-12-10,140859,7908819,467,[1],652,Saul Bellow,bellow,male,1915-06-10
22,1905,Peace,The Nobel Peace Prize,1905-12-10,138089,7753291,468,[1],535,Elizabeth Williams,williams,female,1943-05-22
27,1906,Peace,The Nobel Peace Prize,1906-12-10,138536,7556149,470,[1],105,Burton Richter,richter,male,1931-03-22
32,1907,Peace,The Nobel Peace Prize,1907-12-10,138796,7161123,471,"[1/2, 1/2]",106,Samuel Chao Chung Ting,ting,male,1936-01-27
32,1907,Peace,The Nobel Peace Prize,1907-12-10,138796,7161123,472,"[1/2, 1/2]",409,Baruch S. Blumberg,blumberg,male,1925-07-28
